In [1]:
import requests
import os
import json

In [2]:
import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, ForeignKey, Boolean, Float
from sqlalchemy.orm import relationship
from sqlalchemy.schema import Index
from sqlalchemy import and_, or_, not_

In [3]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, ForeignKey, Boolean, Float
from sqlalchemy.orm import relationship
from sqlalchemy.schema import Index

Base = declarative_base()


class Edge(Base):
    """
    An edge between two nodes in SCADS.
    See https://github.com/commonsense/conceptnet5/wiki/Edges

    key: The unique key of edge in 'edges' table
    relation_key: The key of the relation expressed by the edge. A foreign key from 'relations' table
    start_node_key: The node at the start of the edge. A foreign key from 'nodes' table
    end_node_key: The node at the end of the edge. A foreign key from 'nodes' table
    weight: The weight of the edge
    info: Additional information about the edge
    """
    __tablename__ = 'edges'
    id = Column(Integer, primary_key=True)
    relation_type = Column(Integer, ForeignKey('relations.id'))
    weight = Column(Float)
    start_node = Column(Integer, ForeignKey('nodes.id'))
    end_node = Column(Integer, ForeignKey('nodes.id'))

    relation = relationship("Relation")

    __table_args__ = (
        Index('idx_edges_start_end', "start_node", "end_node"),
        Index('idx_edges_end_start', "end_node", "start_node")
    )

    def __repr__(self):
        return "<Edge(key='%s', relation='%s', weight='%s', start_node='%s', end_node='%s')>" % \
               (self.id, self.relation_type, self.weight, self.start_node, self.end_node)


class Node(Base):
    """
    A node in SCADS.

    For example:
        node = Node(key='0', name='/c/en/...')
    key: The unique key of node in 'nodes' table
    name: The name of the node
    """
    __tablename__ = 'nodes'
    id = Column(Integer, primary_key=True)
    conceptnet_id = Column(String, index=True, unique=True)

    images = relationship("Image", back_populates="node")
    clips = relationship("Clip", back_populates="node")
    outgoing_edges = relationship("Edge", primaryjoin=id == Edge.start_node)

    def __repr__(self):
        return "<Node(key='%s', name='%s'')>" % (self.id, self.conceptnet_id)


class Relation(Base):
    """
    A relation between two nodes in SCADS.
    See https://github.com/commonsense/conceptnet5/wiki/Relations.

    key: The unique key of relation type in 'relations' table
    name: The name of the relation
    type: The type of relation between two nodes. For example 'HasA', 'IsA', and 'Antonym'.
    """
    __tablename__ = 'relations'
    id = Column(Integer, primary_key=True)
    type = Column(String)
    is_directed = Column(Boolean)

    edges = relationship("Edge", back_populates="relation")

    def __repr__(self):
        return "<Relation(key='%s', type='%s', is_directed='%s')>" % (self.id, self.type, self.is_directed)


class Dataset(Base):
    """
    A dataset in SCADS.

    We could have several datasets such as ImageNet, CIFAR10, CIFAR100, MNIST, etc. in SCADS.
    For example:
        cifar100 = Dataset(key=0, name='CIFAR100', nb_classes=100)
    key: The unique, autoincremented key of the dataset
    name: The name of the dataset (optional)
    path: The path to the dataset relative to the SCADS root path (optional)
    """
    __tablename__ = 'datasets'
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String)
    path = Column(String)
    
    images = relationship("Image", back_populates="dataset")
    clips = relationship("Clip", back_populates="dataset")
    
    def __repr__(self):
        return "<Dataset(name='%s')>" % self.name


class Image(Base):
    """
    An Image in SCADS.

    For example:
          apple_img_1 = Image(dataset_key='cifar100 key',
                              node_key='apple key',
                              mode = 'train',
                              location='directory/apple_1.png')

    key: The unique, autoincremented key of the image
    dataset_key: The key of the corresponding dataset for the image. A foreign key from 'datasets' table
    node_key: The key of the corresponding node for the image. A foreign key from 'nodes' table
    mode: If the image is saved for 'train' or 'test' in actual dataset
          If None, the image is not categorized based on train and test
    location: The location of the image relateive to the SCADS root path
    """
    __tablename__ = 'images'
    id = Column(Integer, primary_key=True, autoincrement=True)
    dataset_id = Column(Integer, ForeignKey('datasets.id'))
    node_id = Column(Integer, ForeignKey('nodes.id'), index=True)
    path = Column(String)
    
    dataset = relationship("Dataset", back_populates="images")
    node = relationship("Node", back_populates="images")

    def __repr__(self):
        return "<Image(dataset='%s', node='%s', path='%s'')>" % (self.dataset_id,
                                                                 self.node_id,
                                                                 self.path)


class Clip(Base):
    __tablename__ = "clips"
    id = Column(Integer, primary_key=True, autoincrement=True)
    clip_id = Column(Integer, index=True)
    video_id = Column(Integer)
    base_path = Column(String)
    start_frame = Column(Integer)
    end_frame = Column(Integer)
    real_label = Column(String)

    dataset_id = Column(Integer, ForeignKey('datasets.id'))
    node_id = Column(Integer, ForeignKey('nodes.id'), index=True)

    dataset = relationship("Dataset", back_populates="clips")
    node = relationship("Node", back_populates="clips")

    def __repr__(self):
        return "<Clip(id='%s', node='%s', base_path='%s', start_frame='%s', end_frame='%s')>" % (
            self.id,
            self.node_id,
            self.base_path,
            self.start_frame,
            self.end_frame
        )

In [4]:
#path_to_database = 'predefined/scads.spring2021.sqlite3'
path_to_database = 'test/test_data/scads/test_scads.db'

In [5]:
#Image.__table__.drop(engine)
#Dataset.__table__.drop(engine)
#Relation.__table__.drop(engine)
#Node.__table__.drop(engine)
#Edge.__table__.drop(engine)
#Clip.__table__.create(engine)

In [5]:
engine = sa.create_engine('sqlite:///' + path_to_database)
Session = sessionmaker(bind=engine)
session = Session()

In [7]:
#Clip.__table__.drop(engine)
#Clip.__table__.create(engine)
#session.commit()

In [8]:
#PROBLEM: flic_flac corresponds to two ids

In [6]:
engine.table_names()

['clips', 'datasets', 'edges', 'images', 'nodes', 'relations']

In [47]:
rows = session.query(Image).count()

In [48]:
rows

19

In [6]:
# HMDB
node = session.query(Node).filter(Node.conceptnet_id == '/c/en/jet_ski').first()

In [7]:
node is None

False

In [8]:
node.id

30

In [9]:
len(node.clips)

101

In [10]:
node.clips

[<Clip(id='1021', node='30', base_path='ucf101/ucf101_full/train', start_frame='23470', end_frame='23803')>,
 <Clip(id='1022', node='30', base_path='ucf101/ucf101_full/train', start_frame='23470', end_frame='23803')>,
 <Clip(id='1230', node='30', base_path='ucf101/ucf101_full/train', start_frame='67220', end_frame='67429')>,
 <Clip(id='1705', node='30', base_path='ucf101/ucf101_full/train', start_frame='169727', end_frame='169935')>,
 <Clip(id='1779', node='30', base_path='ucf101/ucf101_full/train', start_frame='185511', end_frame='185719')>,
 <Clip(id='1793', node='30', base_path='ucf101/ucf101_full/train', start_frame='188766', end_frame='188978')>,
 <Clip(id='2152', node='30', base_path='ucf101/ucf101_full/train', start_frame='265569', end_frame='265778')>,
 <Clip(id='2348', node='30', base_path='ucf101/ucf101_full/train', start_frame='303208', end_frame='303395')>,
 <Clip(id='2391', node='30', base_path='ucf101/ucf101_full/train', start_frame='314315', end_frame='314523')>,
 <Clip(

In [40]:
node.clips[-10].real_label

IndexError: list index out of range

In [41]:
node.clips[0].id

IndexError: list index out of range

In [23]:
node.clips[0]

<Clip(id='1021', node='30', base_path='ucf101/ucf101_full/train', start_frame='23470', end_frame='23803')>

In [24]:
len(node.clips)

101

In [42]:
len(node.images)

0

In [11]:
datasets = list(set(c.dataset.name for c in node.clips))
datasets

['UCF101']

In [14]:
datasets = list(set(c.dataset.name for c in node.images))
datasets

['MarsSurface']

In [26]:
# HMDB
node1 = session.query(Node).filter(Node.conceptnet_id == '/c/en/playing').first()
node2 = session.query(Node).filter(Node.conceptnet_id == '/c/en/dhol').first()

In [27]:
len(node1.clips)

1267

In [28]:
len(node2.clips)

328

In [13]:
nodes = ['/c/en/playing', '/c/en/dhol']
objects_node = [session.query(Node).filter(Node.conceptnet_id == n).first() for n in nodes]

In [15]:
objects_node[0].id

31

In [30]:
query = ' '.join([f'nodes.id = {obj.id} OR' \
                  for obj in objects_node[:-1]])
query = ' '.join([query, f'nodes.id = {objects_node[-1].id}'])

In [31]:
query

'nodes.id = 31 OR nodes.id = 32'

In [32]:
q = "SELECT DISTINCT base_path, start_frame, end_frame, clip_id, video_id from clips " \
            "JOIN nodes ON clips.node_id=nodes.id " \
            "WHERE " + query + ";"


In [33]:
results = session.connection().execute(q)


In [34]:
list_res = [x for x in results]

In [35]:
len(list_res)

1267

In [42]:
q = "SELECT DISTINCT images.path from images " \
            "JOIN nodes ON images.node_id = nodes.id " \
            "JOIN datasets ON images.dataset_id = datasets.id " \
            "WHERE nodes.id = " + str(node.id) + ";"

results = session.connection().execute(q)
[x[0] for x in results][-10]

'mars_surface_imgs/mars_surface_imgs_full/test/0571ML0023110000204852E01_DRCL.JPG'

In [ ]:
self.assertEqual(node.get_conceptnet_id(), "/c/en/run")

self.assertEqual(node.get_datasets(images=False), ['HMDB'])

images = node.get_images()
self.assertEqual(len(images), 0)
clips = node.get_clips()
self.assertEqual(len(clips), 19)
self.assertTrue('hmdb/hmdb_full/test'
                in [x[0] for x in clips])
clip = clips[0]
self.assertEqual(clip[0], "hmdb/hmdb_full/train")
self.assertEqual(clip[1], 231465)
self.assertEqual(clip[2], 231565)

In [8]:
#Clip.__table__.drop(engine)
#Clip.__table__.create(engine)

In [9]:
#session.commit()

In [11]:
dataset = session.query(Dataset).filter_by(name='UCF101')#.first()
dataset.delete()
dataset = session.query(Dataset).filter_by(name='HMDB')#.first()
dataset.delete()


session.commit()

In [8]:
# datasets = set()
# data = self.node.images if images else self.node.clips
# for datum in data:
#     datasets.add(datum.dataset.name)
# return list(datasets)

In [35]:
'ShavingBeard'.lower().replace(" ", "_").replace("-", "_")

'shavingbeard'

In [ ]:
ShavingBeard

In [59]:
import re

In [61]:
[i.lower() for i in re.findall('[A-Z][^A-Z]*', ''.join('shaving_beard'.split('_')))]

[]

In [6]:
node = session.query(Node).filter(or_(Node.conceptnet_id.like("/c/en/sword"), 
                                  Node.conceptnet_id.like("/c/en/exercise")))

In [7]:
for n in node:
    print(n)

KeyboardInterrupt: 

In [38]:
data = node.images

AttributeError: 'Query' object has no attribute 'images'

In [17]:
data

[<Image(dataset='4', node='131790', path='domain_net-clipart/domain_net-clipart_full/train/clipart_300_000039.jpg'')>,
 <Image(dataset='4', node='131790', path='domain_net-clipart/domain_net-clipart_full/train/clipart_300_000080.jpg'')>,
 <Image(dataset='4', node='131790', path='domain_net-clipart/domain_net-clipart_full/train/clipart_300_000098.jpg'')>,
 <Image(dataset='4', node='131790', path='domain_net-clipart/domain_net-clipart_full/train/clipart_300_000026.jpg'')>,
 <Image(dataset='4', node='131790', path='domain_net-clipart/domain_net-clipart_full/train/clipart_300_000070.jpg'')>,
 <Image(dataset='4', node='131790', path='domain_net-clipart/domain_net-clipart_full/train/clipart_300_000032.jpg'')>,
 <Image(dataset='4', node='131790', path='domain_net-clipart/domain_net-clipart_full/train/clipart_300_000103.jpg'')>,
 <Image(dataset='4', node='131790', path='domain_net-clipart/domain_net-clipart_full/train/clipart_300_000012.jpg'')>,
 <Image(dataset='4', node='131790', path='domain

In [39]:
data = node.clips

AttributeError: 'Query' object has no attribute 'clips'

In [19]:
data

[<Clip(id='28', node='131790', base_path='external/hmdb/external/hmdb/_full/train', start_frame='2572', end_frame='2617')>,
 <Clip(id='37', node='131790', base_path='external/hmdb/external/hmdb/_full/train', start_frame='3178', end_frame='3256')>,
 <Clip(id='50', node='131790', base_path='external/hmdb/external/hmdb/_full/train', start_frame='4761', end_frame='4825')>,
 <Clip(id='163', node='131790', base_path='external/hmdb/external/hmdb/_full/train', start_frame='16901', end_frame='16947')>,
 <Clip(id='166', node='131790', base_path='external/hmdb/external/hmdb/_full/train', start_frame='17137', end_frame='17215')>,
 <Clip(id='219', node='131790', base_path='external/hmdb/external/hmdb/_full/train', start_frame='21170', end_frame='21217')>,
 <Clip(id='261', node='131790', base_path='external/hmdb/external/hmdb/_full/train', start_frame='25155', end_frame='25179')>,
 <Clip(id='371', node='131790', base_path='external/hmdb/external/hmdb/_full/train', start_frame='35194', end_frame='352

In [ ]:
for r in Clip.__mapper__.relationships:
    print(r)

In [34]:
datasets = set()
for datum in data:
    datasets.add(datum.dataset.name)

AttributeError: 'NoneType' object has no attribute 'name'

In [32]:
datasets

{'GoogleOpenImage'}

In [8]:
brush 35767 616205 616350 6598
hair 42923 616205 616350 6598

SyntaxError: invalid syntax (<ipython-input-8-b7065921fc9b>, line 1)

In [28]:
Image.__table__.columns.keys()

['id', 'dataset_id', 'node_id', 'path']

In [7]:
for row in session.query(Image).filter_by(node_id = '20538').limit(10):
    print(row)

<Image(dataset='4', node='20538', path='domain_net-clipart/domain_net-clipart_full/train/clipart_065_000034.jpg'')>
<Image(dataset='4', node='20538', path='domain_net-clipart/domain_net-clipart_full/train/clipart_065_000038.jpg'')>
<Image(dataset='4', node='20538', path='domain_net-clipart/domain_net-clipart_full/train/clipart_065_000013.jpg'')>
<Image(dataset='4', node='20538', path='domain_net-clipart/domain_net-clipart_full/train/clipart_065_000004.jpg'')>
<Image(dataset='4', node='20538', path='domain_net-clipart/domain_net-clipart_full/train/clipart_065_000012.jpg'')>
<Image(dataset='4', node='20538', path='domain_net-clipart/domain_net-clipart_full/train/clipart_065_000024.jpg'')>
<Image(dataset='4', node='20538', path='domain_net-clipart/domain_net-clipart_full/train/clipart_065_000035.jpg'')>
<Image(dataset='4', node='20538', path='domain_net-clipart/domain_net-clipart_full/train/clipart_065_000026.jpg'')>
<Image(dataset='4', node='20538', path='domain_net-clipart/domain_net-cl

In [7]:
for row in session.query(Clip).filter_by(node_id = '18371').limit(10):
    print(row)

<Clip(id='30875', node='18371', base_path='external/hmdb/external/hmdb/_full/train', start_frame='13758', end_frame='13948'
<Clip(id='30917', node='18371', base_path='external/hmdb/external/hmdb/_full/train', start_frame='17093', end_frame='17136'
<Clip(id='31089', node='18371', base_path='external/hmdb/external/hmdb/_full/train', start_frame='27712', end_frame='28035'
<Clip(id='31228', node='18371', base_path='external/hmdb/external/hmdb/_full/train', start_frame='36740', end_frame='36797'
<Clip(id='31436', node='18371', base_path='external/hmdb/external/hmdb/_full/train', start_frame='51564', end_frame='52034'
<Clip(id='31440', node='18371', base_path='external/hmdb/external/hmdb/_full/train', start_frame='52206', end_frame='52329'
<Clip(id='31464', node='18371', base_path='external/hmdb/external/hmdb/_full/train', start_frame='53871', end_frame='54163'
<Clip(id='31588', node='18371', base_path='external/hmdb/external/hmdb/_full/train', start_frame='62826', end_frame='62920'
<Clip(id

In [8]:
session.query(Node).filter_by(conceptnet_id ='/c/en/bow').all()

[<Node(key='18371', name='/c/en/bow'')>]

In [13]:
results = session.query(Clip).filter_by(clip_id==6598).all()#limit(100)

TypeError: filter_by() takes 1 positional argument but 2 were given

In [12]:
for row in results:
    print(row)

<Clip(id='22420', node='4890438', base_path='external/hmdb/external/hmdb/_full/train', start_frame='411976', end_frame='412056'


In [9]:
#dataset = session.query(Dataset).filter_by(name='HMDB').first()

In [10]:
#dataset

# Try API

In [8]:
import requests
import os
import json

In [9]:
secret = 'a5aed2a8-db80-4b22-bf72-11f2d0765572'
url = 'https://api-dev.lollllz.com/'

headers = {'user_secret': secret, 'govteam_secret': 'mock'}
r = requests.get(f"{url}/list_tasks", headers=headers)
r.json()

{'tasks': ['06023f86-a66b-4b2c-8b8b-951f5edd0f22',
  '3159d8ef-dfef-48fc-9325-829258be5f28',
  '6d5e1f85-5d8f-4cc9-8184-299db03713f4',
  '923cbe9c-1e4f-4a05-a156-dc972ef5edf5',
  '9c103cc4-e2e1-4070-9877-c3b64a6f327f',
  '9c391ced-19ed-489a-9be0-ecf11fc4dcab',
  'a1182934-69d2-4a46-bcbb-feab912cce2c',
  'b01a6738-0b85-46c2-9318-16c3e2ef0f6d',
  'bbfadb2c-c7c3-4596-b548-3dd01a6d1d2c',
  'problem_test_image_classification',
  'problem_test_machine_translation',
  'problem_test_obj_detection',
  'problem_test_video_classification']}

In [11]:
headers = {'user_secret': secret, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}
r = requests.get(f"{url}/task_metadata/9c103cc4-e2e1-4070-9877-c3b64a6f327f", headers=headers)
print(json.dumps(r.json(), indent=2))

NameError: name 'secret' is not defined

In [ ]:

  
  '6d5e1f85-5d8f-4cc9-8184-299db03713f4',(CIFAR)
  '9c103cc4-e2e1-4070-9877-c3b64a6f327f',(Video)
  '9c391ced-19ed-489a-9be0-ecf11fc4dcab', (MARS)
  'b01a6738-0b85-46c2-9318-16c3e2ef0f6d', (domain net)
  'bbfadb2c-c7c3-4596-b548-3dd01a6d1d2c', (domain clipart)

In [40]:
headers = {'user_secret': secret, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

# This is a convenience for development purposes, IN EVAL ALWAYS USE `full`
data_type = 'sample' # can either be `sample` or `full`

# Option to customize the session name 
r = requests.post(f"{url}/auth/create_session", json={ 'data_type': data_type, 
                                                      'task_id': '9c391ced-19ed-489a-9be0-ecf11fc4dcab'},
                  headers=headers)
r.json()

{'session_token': 'GOyDDTjOQn6eL1uvuzdT'}

In [41]:
session_token = r.json()['session_token']

In [42]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.get(f"{url}/session_status", headers=headers)
r.json()

{'Session_Status': {'active': 'In Progress',
  'budget_left_until_checkpoint': 23,
  'budget_used': 0,
  'current_dataset': {'classes': ['apxs',
    'apxs cal target',
    'chemcam cal target',
    'chemin inlet open',
    'drill',
    'drill holes',
    'drt front',
    'drt side',
    'ground',
    'horizon',
    'inlet',
    'mahli',
    'mahli cal target',
    'mastcam',
    'mastcam cal target',
    'observation tray',
    'portion box',
    'portion tube',
    'portion tube opening',
    'rems uv sensor',
    'rover rear deck',
    'scoop',
    'sun',
    'turret',
    'wheel'],
   'dataset_type': 'image_classification',
   'license_citation': '10.5281/zenodo.1049137. Please also cite this paper, which provides additional details about the                                  data set. Kiri L. Wagstaff, You Lu, Alice Stanboli, Kevin Grimes, Thamme Gowda, and Jordan Padams. "Deep Mars: CNN                                  Classification of Mars Imagery for the PDS Imaging Atlas."\xa0P

In [45]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.get(f"{url}/task_metadata/9c391ced-19ed-489a-9be0-ecf11fc4dcab", headers=headers)
print(json.dumps(r.json(), indent=2))

{
  "task_metadata": {
    "adaptation_dataset": "msl_curiosity_imgs",
    "adaptation_evaluation_metrics": [
      "accuracy"
    ],
    "adaptation_label_budget_full": [
      19,
      38,
      76,
      152,
      384,
      972,
      2459,
      6220
    ],
    "adaptation_label_budget_sample": [
      19,
      38,
      76,
      152,
      161,
      170,
      180,
      190
    ],
    "base_dataset": "mars_surface_imgs",
    "base_evaluation_metrics": [
      "accuracy"
    ],
    "base_label_budget_full": [
      25,
      50,
      100,
      200,
      455,
      1037,
      2362,
      5380
    ],
    "base_label_budget_sample": [
      23,
      46,
      92,
      184,
      195,
      206,
      218,
      230
    ],
    "problem_type": "image_classification",
    "task_id": "9c391ced-19ed-489a-9be0-ecf11fc4dcab",
    "whitelist": [
      "imagenet_1k",
      "coco2014",
      "voc2009",
      "imagenet_bbox",
      "cifar100",
      "domain_net-quickdraw",
      "do

In [107]:
len(r.json()['Session_Status']['current_dataset']['classes'])

42

In [98]:
r.json()['Session_Status']['current_dataset']['classes']

['lollipop',
 'apple',
 'diamond',
 'helmet',
 'skull',
 'palm_tree',
 'lipstick',
 'cat',
 'rhinoceros',
 'peanut',
 'animal_migration',
 'pond',
 'ant',
 'fire_hydrant',
 'jacket',
 'blueberry',
 'microwave',
 'remote_control',
 'tree',
 'paintbrush',
 'butterfly',
 'see_saw',
 'crown',
 'leaf',
 'boomerang',
 'drill',
 'toaster',
 'lightning',
 'toe',
 'garden_hose',
 'sword',
 'fork',
 'pear',
 'hand',
 'fireplace',
 'sandwich',
 'strawberry',
 'raccoon',
 'bench',
 'ice_cream',
 'piano',
 'basket',
 'chandelier',
 'elbow',
 'sun',
 'cactus',
 'car',
 'crab',
 'cello',
 'peas',
 'pig',
 'hot_air_balloon',
 'tractor',
 'hammer',
 'ocean',
 'canoe',
 'screwdriver',
 'river',
 'feather',
 'snail',
 'eye',
 'bed',
 'violin',
 'golf_club',
 'tooth',
 'diving_board',
 'yoga',
 'hockey_stick',
 'rain',
 'cup',
 'calendar',
 'stereo',
 'radio',
 'angel',
 'trombone',
 'snowman',
 'sweater',
 'microphone',
 'aircraft_carrier',
 'calculator',
 'camouflage',
 'shovel',
 'string_bean',
 'telev

In [99]:
headers = {'user_secret': secret, 'session_token': session_token, 'govteam_secret': os.environ.get('GOVTEAM_SECRET')}

r = requests.get(f"{url}/seed_labels", headers=headers)
results = r.json()#['Labels']

In [100]:
len(results['Labels'])

345

In [83]:
results['Labels']

[{'class': ' apxs', 'id': '0173MR0008910000201795I01_DRCL.JPG'},
 {'class': ' apxs cal target', 'id': '0179MH0002170010102506C00_DRCL.JPG'},
 {'class': ' chemcam cal target', 'id': '0025ML0001270080100815E01_DRCL.JPG'},
 {'class': ' chemin inlet open', 'id': '0036MH0000510030100053C00_DRCL.JPG'},
 {'class': ' drill', 'id': '0128MR0005340000200813E01_DRCL.JPG'},
 {'class': ' drill holes', 'id': '0180MH0001490020102527E01_DRCL.JPG'},
 {'class': ' drt side', 'id': '0169MR0009110000201658I01_DRCL.JPG'},
 {'class': ' ground', 'id': '0072MR0005610170103642E01_DRCL.JPG'},
 {'class': ' horizon', 'id': '0003ML0000000160100036I01_DRCL.JPG'},
 {'class': 'inlet', 'id': '0163ML0008760040104601I01_DRCL.JPG'},
 {'class': 'mahli', 'id': '0032MR0000760020100776I01_DRCL.JPG'},
 {'class': 'mahli cal target', 'id': '0034MH0000470040100046I01_DRCL.JPG'},
 {'class': 'mastcam', 'id': '0177MH0002260000102364C00_DRCL.JPG'},
 {'class': 'mastcam cal target', 'id': '0013MR0000020060100034D01_DRCL.JPG'},
 {'class'